# SQL 的五十道練習

> 條件邏輯

[數據交點](https://www.datainpoint.com) | 郭耀仁 <yaojenkuo@datainpoint.com>

## 練習題指引

- 在每份練習題的開始，都會先將四個學習資料庫載入環境。
- 因此 SQL 可以指定四個學習資料庫中的資料表，不需要額外指定資料庫。
- 在 SQL 語法起點與 SQL 語法終點這兩個單行註解之間撰寫能夠得到預期結果的 SQL。
- 可以先在自己電腦的 SQLiteStudio 或者 DBeaver 寫出跟預期結果相同的 SQL 後再複製貼上到練習題。
- 執行測試的方式為點選上方選單的 Kernel -> Restart & Run All -> Restart and Run All Cells。
- 可以每寫一題就執行測試，也可以全部寫完再執行測試。
- 練習題閒置超過 10 分鐘會自動斷線，這時只要重新點選練習題連結即可重新啟動。

In [1]:
import sqlite3
import unittest
import json
import os
import numpy as np
import pandas as pd
conn = sqlite3.connect('../databases/nba.db')
conn.execute("""ATTACH '../databases/covid19.db' AS covid19""")
conn.execute("""ATTACH '../databases/twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH '../databases/imdb.db' AS imdb""")

## 22. 從 `covid19` 資料庫的 `daily_report` 資料表將「美國」與「非美國」的觀測值用衍生計算欄位區分，美國的觀測值給予整數 1、非美國的觀測值給予整數 0，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(3981, 2) 的查詢結果。

```
                 Combined_Key  is_us
0                 Afghanistan      0
1                     Albania      0
2                     Algeria      0
3                     Andorra      0
4                      Angola      0
...                       ...    ...
3976       Teton, Wyoming, US      1
3977       Uinta, Wyoming, US      1
3978  Unassigned, Wyoming, US      1
3979    Washakie, Wyoming, US      1
3980      Weston, Wyoming, US      1

[3981 rows x 2 columns]
```

In [2]:
case_is_us_from_daily_report =\
"""
-- SQL 查詢語法起點
SELECT Combined_Key,
       CASE WHEN Combined_Key LIKE '%, US' THEN 1
            ELSE 0 END AS is_us
  FROM daily_report
 ORDER BY is_us;
-- SQL 查詢語法終點
"""

## 23. 從 `imdb` 資料庫的 `movies` 資料表將評等超過 8.7（`>8.7`）的電影分類為 `'Awesome'`、將評等超過 8.4（`>8.4`）的電影分類為 `'Terrific'`，再將其餘的電影分類為 `'Great'`，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(250, 3) 的查詢結果。

```
                                              title  rating rating_category
0                          The Shawshank Redemption     9.3         Awesome
1                                     The Godfather     9.2         Awesome
2                            The Godfather: Part II     9.0         Awesome
3                                   The Dark Knight     9.0         Awesome
4                                      12 Angry Men     9.0         Awesome
..                                              ...     ...             ...
245  Neon Genesis Evangelion: The End of Evangelion     8.1           Great
246                              7 Kogustaki Mucize     8.2           Great
247                                      Tangerines     8.2           Great
248                                        Drishyam     8.2           Great
249                                          Swades     8.2           Great

[250 rows x 3 columns]
```

In [3]:
case_rating_category_from_movies =\
"""
-- SQL 查詢語法起點
SELECT title,
       rating,
       CASE WHEN rating > 8.7 THEN 'Awesome'
            WHEN rating > 8.4 THEN 'Terrific'
            ELSE 'Great' END AS rating_category
  FROM movies;
-- SQL 查詢語法終點
"""

## 24. 從 `twElection2020` 資料庫的 `admin_regions` 資料表將 `county` 分類為 `'六都'`與`'非六都'`，參考下列的預期查詢結果。

PS 六都為臺北市、新北市、桃園市、臺中市、臺南市與高雄市。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(22, 2) 的查詢結果。

```
   county county_type
0     新北市          六都
1     桃園市          六都
2     臺中市          六都
3     臺北市          六都
4     臺南市          六都
5     高雄市          六都
6     南投縣         非六都
7     嘉義市         非六都
8     嘉義縣         非六都
9     基隆市         非六都
10    宜蘭縣         非六都
11    屏東縣         非六都
12    彰化縣         非六都
13    新竹市         非六都
14    新竹縣         非六都
15    澎湖縣         非六都
16    臺東縣         非六都
17    花蓮縣         非六都
18    苗栗縣         非六都
19    連江縣         非六都
20    金門縣         非六都
21    雲林縣         非六都
```

In [4]:
case_county_type_from_admin_regions =\
"""
-- SQL 查詢語法起點
SELECT DISTINCT county,
       CASE WHEN county IN ('臺北市', '新北市', '桃園市', '臺中市', '臺南市', '高雄市') THEN '六都'
            ELSE '非六都' END AS county_type
  FROM admin_regions
 ORDER BY county_type,
          county;
-- SQL 查詢語法終點
"""

## 執行測試！

Kernel -> Restart & Run All -> Restart and Run All Cells.

In [5]:
class TestCase(unittest.TestCase):
    def test_22_case_is_us_from_daily_report(self):
        is_us_from_daily_report = pd.read_sql(case_is_us_from_daily_report, conn)
        self.assertEqual(is_us_from_daily_report.shape, (3981, 2))
        column_values = set(is_us_from_daily_report.iloc[:, 1].values)
        self.assertTrue(0 in column_values)
        self.assertTrue(1 in column_values)
    def test_23_case_rating_category_from_movies(self):
        rating_category_from_movies = pd.read_sql(case_rating_category_from_movies, conn)
        self.assertEqual(rating_category_from_movies.shape, (250, 3))
        column_values = set(rating_category_from_movies.iloc[:, 2].values)
        self.assertTrue('Awesome' in column_values)
        self.assertTrue('Terrific' in column_values)
        self.assertTrue('Great' in column_values)
    def test_24_case_county_type_from_admin_regions(self):
        county_type_from_admin_regions = pd.read_sql(case_county_type_from_admin_regions, conn)
        self.assertEqual(county_type_from_admin_regions.shape, (22, 2))
        column_values = set(county_type_from_admin_regions.iloc[:, 1].values)
        column_values_arr = county_type_from_admin_regions.iloc[:, 1].values
        self.assertTrue('六都' in column_values)
        self.assertTrue('非六都' in column_values)
        self.assertEqual(column_values_arr[column_values_arr == '六都'].size, 6)
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestCase)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)
cwd = os.getcwd()
folder_name = cwd.split("/")[-1]
with open("../exercise_index.json", "r") as content:
    exercise_index = json.load(content)
chapter_name = exercise_index[folder_name]

test_22_case_is_us_from_daily_report (__main__.TestCase) ... ok
test_23_case_rating_category_from_movies (__main__.TestCase) ... ok
test_24_case_county_type_from_admin_regions (__main__.TestCase) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.153s

OK


In [6]:
print("您在「{}」章節中的 {} 道 SQL 練習答對了 {} 題。".format(chapter_name, number_of_test_runs, number_of_successes))

您在「條件邏輯」章節中的 3 道 SQL 練習答對了 3 題。
